# Regresión mpg

Regresión con pyspark para predecir columna mpg del dataset mpg.

Importante el uso de VectorAssembler.

Por ejemplo, en un DataFrame de Spark, la librería no espera que tengas 4, 5 o 6 columnas separadas para las variables explicativas, sino una sola columna del tipo Vector que contenga esos 4, 5 o 6 valores.

Spark provee VectorAssembler, que usa varias columnas numéricas (o categóricas transformadas) y las combina en un solo vector llamado "features".

Internamente, VectorAssembler crea (por cada fila) un DenseVector o SparseVector con todas las columnas que le pasas. Así, en la etapa de entrenamiento, el modelo lee esa columna vectorial y la columna de label (la que quieres predecir).

DataFrames:

* pyspark.sql.DataFrame (el DataFrame "real" de Spark):
    * Este es el tipo que entienden todos los métodos de ML de PySpark (.ml o .mllib).
    * Permite paralelizar operaciones, transformaciones, etc. en un clúster Spark.
    * Tiene sus propios métodos (.select, .withColumn, .show, etc.).
    * Tenemos que crear column a features y label para trabajar con los modelos de MLlib habitualmente.

* pyspark.pandas.DataFrame (o ps.DataFrame):

    * Es un DataFrame con API parecida a la de pandas (intenta imitar la sintaxis de pandas).
    * Por debajo, se apoya en Spark, pero no es directamente compatible con la librería ML.
    * Existe para hacer más fácil la transición a Spark de un usuario que maneje pandas, pero no permite usar los algoritmos de Spark ML directamente.

In [25]:
import seaborn as sns
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('regresion_mpg').getOrCreate()
df = spark.createDataFrame(sns.load_dataset('mpg').dropna()) # Le quitamos los nulos
df.show(5)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|                name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|   usa|chevrolet chevell...|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|   usa|   buick skylark 320|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|   usa|  plymouth satellite|
|16.0|        8|       304.0|     150.0|  3433|        12.0|        70|   usa|       amc rebel sst|
|17.0|        8|       302.0|     140.0|  3449|        10.5|        70|   usa|         ford torino|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 5 rows



In [26]:
# opción 1: hacer assembler antes de particionar datos
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year'],
    outputCol='features' # le llamamos features para que coincida con lo que piden los algoritmos
)
df_assembled = assembler.transform(df)
df_assembled.show(3)

+----+---------+------------+----------+------+------------+----------+------+--------------------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|                name|            features|
+----+---------+------------+----------+------+------------+----------+------+--------------------+--------------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|   usa|chevrolet chevell...|[8.0,307.0,130.0,...|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|   usa|   buick skylark 320|[8.0,350.0,165.0,...|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|   usa|  plymouth satellite|[8.0,318.0,150.0,...|
+----+---------+------------+----------+------+------------+----------+------+--------------------+--------------------+
only showing top 3 rows



In [27]:
df_features_label = df_assembled.withColumnRenamed('mpg', 'label').select('features', 'label')
df_features_label.show(3)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[8.0,307.0,130.0,...| 18.0|
|[8.0,350.0,165.0,...| 15.0|
|[8.0,318.0,150.0,...| 18.0|
+--------------------+-----+
only showing top 3 rows



In [28]:
df_train, df_test = df_features_label.randomSplit([0.8, 0.2], seed=42)

In [29]:
# Opción 2: primero particionar y luego usar VectorAssembler

numeric_cols = ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']
label_col = 'mpg'

df_selected = df.select(numeric_cols + [label_col])
# df_selected.show(1)

df_train, df_test = df_selected.randomSplit([0.8, 0.2], seed=42)

assembler = VectorAssembler(
    inputCols=numeric_cols,
    outputCol='features'
)
# Le hemos dejado como nombre 'mpg' en lugar de 'label' para ver
# cómo usarlo en los algoritmos ML
df_train = assembler.transform(df_train).select('features', label_col)
df_test = assembler.transform(df_test).select('features', label_col)

In [30]:
from pyspark.ml.regression import LinearRegression
# como no lo hemos renombrado a label tenemos que poner su nombre completo
lr = LinearRegression(labelCol=label_col)
model = lr.fit(df_train)
df_pred = model.transform(df_test)
df_pred.show(3)

+--------------------+----+------------------+
|            features| mpg|        prediction|
+--------------------+----+------------------+
|[4.0,104.0,95.0,2...|25.0|22.759564401576107|
|[4.0,121.0,113.0,...|26.0| 23.26698094028353|
|[6.0,199.0,97.0,2...|18.0| 20.16496724690829|
+--------------------+----+------------------+
only showing top 3 rows



In [32]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator_r2 = RegressionEvaluator(metricName='r2', labelCol=label_col)
evaluator_mae = RegressionEvaluator(metricName='mae', labelCol=label_col)
evaluator_mse = RegressionEvaluator(metricName='mse', labelCol=label_col)
evaluator_rmse = RegressionEvaluator(metricName='rmse', labelCol=label_col)

print('r2', evaluator_r2.evaluate(df_pred))
print('mae', evaluator_mae.evaluate(df_pred))
print('mse', evaluator_mse.evaluate(df_pred))
print('rmse', evaluator_rmse.evaluate(df_pred))

r2 0.8008919249079934
mae 2.496475133849499
mse 10.002107194842727
rmse 3.1626108193773588


In [34]:
# detectar nombres de columnas numéricas automáticamente, select_dtypes de pandas:
from pyspark.sql.types import NumericType 

numeric_cols = [field.name for field in df.schema.fields if isinstance(field.dataType, NumericType)]
numeric_cols

['mpg',
 'cylinders',
 'displacement',
 'horsepower',
 'weight',
 'acceleration',
 'model_year']

In [36]:
df.select(numeric_cols).show(1)

+----+---------+------------+----------+------+------------+----------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|
+----+---------+------------+----------+------+------------+----------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|
+----+---------+------------+----------+------+------------+----------+
only showing top 1 row



In [38]:
from pyspark.sql.types import StringType 

categorical_cols = [field.name for field in df.schema.fields if isinstance(field.dataType, StringType)]
df_categorical = df.select(categorical_cols)
df_categorical.show(1)

+------+--------------------+
|origin|                name|
+------+--------------------+
|   usa|chevrolet chevell...|
+------+--------------------+
only showing top 1 row

